In [82]:
import pandas as pd

fileGenre = open('../data/u.genre', 'r')
fileItem = open('../data/u.item', 'r')
fileUser = open('../data/u.user', 'r')

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        


        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gendre'] = rec[2]
            item['function'] = rec[4]
        else:
            return

        data.append(item)
    filed.close()
    return data

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)


df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)

In [83]:

fileData = open('../data/u.data', 'r')
def getDataframeFromFileWithSpaces(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

relationship = getDataframeFromFileWithSpaces(fileData)
df_relations = pd.DataFrame(relationship)

In [86]:
df_users
df_movies
df_relations

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [84]:
merged_df = pd.merge(df_movies, df_relations[['userId', 'movieId', 'rating','timestamp']], on='movieId', how='inner')

In [96]:
merged_df[(merged_df['movieId'] == '204') & (merged_df['userId'] == '716')]

,movieId,name,release_date,url,genre,userId,rating,timestamp
31342,204,Back to the Future (1985),01-Jan-1985,http://us.imdb.com/M/title-exact?Back%20to%20t...,"[Comedy, Sci-Fi]",716,5,879795543
